## Import Library

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import math
import timm
import yaml
import random
import torch
import torch.nn as nn
import albumentations
import albumentations.pytorch

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler, CosineAnnealingLR, StepLR
from sklearn.model_selection import StratifiedKFold, train_test_split
from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from easydict import EasyDict
from torchvision import transforms
from PIL import Image

In [2]:
patience = 10
counter = 0

num_epochs = 50
accumulation_steps = 2
batch_size = 64
train_log_interval = 100

LEARNING_RATE = 0.0001 
lr_decay_step = 5

## Bring csv & Delete noise data

In [3]:
filename = 'OpenData_PotOpenTabletIdntfc20220412.xls'
df = pd.read_excel(filename, engine='openpyxl')

In [4]:
df[df['품목일련번호']==201900995]

,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,...,전문일반구분,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자
24116,201900995,트라마펜세미서방정,19640018,구주제약(주),연한 노란색의 원형 서방성 필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,GJ2,NaN,원형,주황,...,전문의약품,20190219,서방성필름코팅정,NaN,NaN,NaN,NaN,NaN,NaN,20220328.0


In [5]:
## delete 구강정 데이터
index_delete1 = df[df['품목일련번호']==200605327].index
index_delete2 = df[df['품목일련번호']==200605328].index
index_delete3 = df[df['품목일련번호']==200605329].index
index_delete4 = df[df['품목일련번호']==200605330].index
index_delete5 = df[df['품목일련번호']==200605331].index
index_delete6 = df[df['품목일련번호']==200606263].index

## delete 반원형 데이터
index_delete7 = df[df['품목일련번호']==197800388].index
index_delete8 = df[df['품목일련번호']==199906868].index
index_delete9 = df[df['품목일련번호']==197900378].index

## delete some strange data
# index_delete10 = df[df['품목일련번호']==200201439].index
# index_delete11 = df[df['품목일련번호']==201401487].index
# index_delete12 = df[df['품목일련번호']==202105657].index
# index_delete13 = df[df['품목일련번호']==201803406].index
# index_delete14 = df[df['품목일련번호']==200100083].index
# index_delete15 = df[df['품목일련번호']==201900995].index

In [6]:
df = df.drop(index_delete1)
df = df.drop(index_delete2)
df = df.drop(index_delete3)
df = df.drop(index_delete4)
df = df.drop(index_delete5)
df = df.drop(index_delete6)
df = df.drop(index_delete7)
df = df.drop(index_delete8)
df = df.drop(index_delete9)
# df = df.drop(index_delete10)
# df = df.drop(index_delete11)
# df = df.drop(index_delete12)
# df = df.drop(index_delete13)
# df = df.drop(index_delete14)
# df = df.drop(index_delete15)

In [7]:
df.iloc[:, [6, 7]].values

array([['IDG', nan],
       ['YH', 'LT'],
       [nan, nan],
       ...,
       ['DTT DTT', nan],
       ['ΛTB', '분할선'],
       ['SK DEX', nan]], dtype=object)

In [8]:
is_text_mark_nan = []
for front, back in df.iloc[:, [6, 7]].values:
    # nan: 0, text: 1, mark: 2
    if type(front) is float and type(back) is float:
        is_text_mark_nan.append(0)
    else:
        if (type(front) is not float and '마크' in front) or (type(back) is not float and '마크' in back):
            is_text_mark_nan.append(2)
        else:
            is_text_mark_nan.append(1)

In [9]:
df.insert(29, 'text_mark_nan', is_text_mark_nan)

In [10]:
num_classes = len(set(is_text_mark_nan))

## Create CustomDataset

In [11]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        super().__init__()
        self.df = df.reset_index()
        self.image_id = self.df['품목일련번호']
        self.labels = self.df['text_mark_nan']
        self.transform = transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        image_id = self.image_id[idx]
        label = self.labels[idx]
        image_path = f'/opt/ml/data_handling/data/{image_id}.jpg'
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        
        return image, label

## Split Train/Val

In [12]:
image_num = df['품목일련번호']
label = df['text_mark_nan']

# https://teddylee777.github.io/scikit-learn/train-test-split
x_train, x_valid, y_train, y_valid = train_test_split(image_num, label, test_size=0.2, stratify=label, random_state=22)

In [13]:
## https://mizykk.tistory.com/131

train_zip = zip(x_train, y_train)
train_df = pd.DataFrame(train_zip)
train_df.columns = ['품목일련번호','text_mark_nan']

val_zip = zip(x_valid, y_valid)
val_df = pd.DataFrame(val_zip)
val_df.columns = ['품목일련번호','text_mark_nan']

In [14]:
train_df.groupby('text_mark_nan').count()

,품목일련번호
text_mark_nan,
0,147
1,16779
2,2564


In [15]:
train_df.groupby('text_mark_nan').count().sum()

품목일련번호    19490
dtype: int64

In [16]:
val_df.groupby('text_mark_nan').count()

,품목일련번호
text_mark_nan,
0,37
1,4195
2,641


In [17]:
val_df.groupby('text_mark_nan').count().sum()

품목일련번호    4873
dtype: int64

In [18]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

train_dataset = CustomDataset(train_df, transform=transform)
val_dataset = CustomDataset(val_df, transform=transform)

In [19]:
image, label = next(iter(train_dataset))
image, label

(tensor([[[1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          ...,
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529]],
 
         [[1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          ...,
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707]],
 
         [[2.1171, 2.1171, 2.1171,  ..., 2.1171, 2.1171, 2.1171],
          [2.1171, 2.1171, 2.1171,  ..., 2.1171, 2.1171, 2.1171],
          [2.1171, 2.1171, 2.1171,  ...,

In [20]:
image, label = next(iter(val_dataset))
image, label

(tensor([[[1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          ...,
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529],
          [1.1529, 1.1529, 1.1529,  ..., 1.1529, 1.1529, 1.1529]],
 
         [[1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          ...,
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707],
          [1.5707, 1.5707, 1.5707,  ..., 1.5707, 1.5707, 1.5707]],
 
         [[2.1171, 2.1171, 2.1171,  ..., 2.1171, 2.1171, 2.1171],
          [2.1171, 2.1171, 2.1171,  ..., 2.1171, 2.1171, 2.1171],
          [2.1171, 2.1171, 2.1171,  ...,

In [21]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)

## Pretrained Model

In [22]:
model_name = 'resnet18'
model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device) 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [23]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 
scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

In [24]:
import wandb
wandb.init(project="final-project", entity="medic", name=f"KM_{model_name}_text_mark_non")

name = f'{model_name}_type_and_shape'
os.makedirs(os.path.join(os.getcwd(), 'results', name), exist_ok=True)

counter = 0
best_val_acc = 0
best_val_loss = np.inf

for epoch in range(num_epochs):
    # train loop
    model.train()
    loss_value = 0
    matches = 0
    for idx, train_batch in tqdm(enumerate(train_loader)):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)

        loss.backward()
        
        # -- Gradient Accumulation
        if (idx+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx + 1) % train_log_interval == 0:
            train_loss = loss_value / train_log_interval
            train_acc = matches / batch_size / train_log_interval
            current_lr = scheduler.get_last_lr()
            print(
                f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
            )

            loss_value = 0
            matches = 0
            
    scheduler.step()

    # val loop
    with torch.no_grad():
        print("Calculating validation results...")
        model.eval()
        val_loss_items = []
        val_acc_items = []
        label_accuracy, total_label = [0]*num_classes, [0]*num_classes
        for val_batch in val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
            ## label별 accuracy
            for i in range(len(labels)):
                total_label[int(labels[i])] += 1
                if labels[i] == preds[i]:
                    label_accuracy[int(labels[i])] += 1
            

        val_loss = np.sum(val_loss_items) / len(val_loader)
        val_acc = np.sum(val_acc_items) / len(val_df)
        
        # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_acc > best_val_acc:
            print("New best model for val accuracy! saving the model..")
            torch.save(model.state_dict(), f"results/{name}/best.ckpt")
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
        if counter > patience:
            print("Early Stopping...")
            break
        
        ## 파이썬 배열 나눗셈 https://bearwoong.tistory.com/60
        accuracy_by_label = np.array(label_accuracy)/np.array(total_label)
        print(f"accuracy by label: {accuracy_by_label}")
        
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        
        print(
            f"nan: {accuracy_by_label[0]}\n"
            f"text: {accuracy_by_label[1]}\n"
            f"mark: {accuracy_by_label[2]}\n"
        )

    wandb.log({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "valid_loss": val_loss,
        "valid_accuracy": val_acc,
        "best_loss": best_val_loss,
        "best_accuracy": best_val_acc,
        "nan": accuracy_by_label[0],
        "text": accuracy_by_label[1],
        "mark": accuracy_by_label[2],
    })

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /opt/ml/.netrc


Epoch[0/50](100/305) || training loss 0.5881 || training accuracy 77.42% || lr [0.0001]
Epoch[0/50](200/305) || training loss 0.4008 || training accuracy 86.30% || lr [0.0001]
Epoch[0/50](300/305) || training loss 0.3679 || training accuracy 86.84% || lr [0.0001]

Calculating validation results...
New best model for val accuracy! saving the model..
accuracy by label: [0.83783784 0.96400477 0.24960998]
[Val] acc : 86.91%, loss: 0.35 || best acc : 86.91%, best loss: 0.35
nan: 0.8378378378378378
text: 0.964004767580453
mark: 0.24960998439937598



Epoch[1/50](100/305) || training loss 0.3208 || training accuracy 88.36% || lr [0.0001]
Epoch[1/50](200/305) || training loss 0.2916 || training accuracy 89.23% || lr [0.0001]
Epoch[1/50](300/305) || training loss 0.2558 || training accuracy 90.78% || lr [0.0001]

Calculating validation results...
New best model for val accuracy! saving the model..
accuracy by label: [0.83783784 0.97783075 0.4149766 ]
[Val] acc : 90.27%, loss: 0.27 || best acc : 90.27%, best loss: 0.27
nan: 0.8378378378378378
text: 0.9778307508939214
mark: 0.41497659906396256



...

Epoch[24/50](100/305) || training loss 0.0003357 || training accuracy 100.00% || lr [6.25e-06]
Epoch[24/50](200/305) || training loss 0.0003808 || training accuracy 100.00% || lr [6.25e-06]
Epoch[24/50](300/305) || training loss 0.0003498 || training accuracy 100.00% || lr [6.25e-06]

Calculating validation results...
accuracy by label: [0.83783784 0.98235995 0.6099844 ]
[Val] acc : 93.23%, loss: 0.42 || best acc : 93.31%, best loss: 0.25
nan: 0.8378378378378378
text: 0.9823599523241955
mark: 0.609984399375975



Epoch[25/50](100/305) || training loss 0.0003918 || training accuracy 100.00% || lr [3.125e-06]
Epoch[25/50](200/305) || training loss 0.0003584 || training accuracy 100.00% || lr [3.125e-06]
Epoch[25/50](300/305) || training loss 0.0003401 || training accuracy 100.00% || lr [3.125e-06]

Calculating validation results...
accuracy by label: [0.86486486 0.98116806 0.61466459]
[Val] acc : 93.21%, loss: 0.42 || best acc : 93.31%, best loss: 0.25
nan: 0.8648648648648649
text: 0.9811680572109654
mark: 0.6146645865834633



Epoch[26/50](100/305) || training loss 0.0003662 || training accuracy 100.00% || lr [3.125e-06]
Epoch[26/50](200/305) || training loss 0.0003521 || training accuracy 100.00% || lr [3.125e-06]
Epoch[26/50](300/305) || training loss 0.0003277 || training accuracy 100.00% || lr [3.125e-06]

Calculating validation results...
accuracy by label: [0.86486486 0.97806913 0.62402496]
[Val] acc : 93.06%, loss: 0.42 || best acc : 93.31%, best loss: 0.25
nan: 0.8648648648648649
text: 0.9780691299165674
mark: 0.62402496099844



Epoch[27/50](100/305) || training loss 0.0003688 || training accuracy 100.00% || lr [3.125e-06]
Epoch[27/50](200/305) || training loss 0.0003796 || training accuracy 100.00% || lr [3.125e-06]
Epoch[27/50](300/305) || training loss 0.0005596 || training accuracy 100.00% || lr [3.125e-06]

Calculating validation results...
accuracy by label: [0.86486486 0.9794994  0.61622465]
[Val] acc : 93.08%, loss: 0.42 || best acc : 93.31%, best loss: 0.25
nan: 0.8648648648648649
text: 0.9794994040524434
mark: 0.6162246489859594



Epoch[28/50](100/305) || training loss 0.0004618 || training accuracy 100.00% || lr [3.125e-06]
Epoch[28/50](200/305) || training loss 0.0002873 || training accuracy 100.00% || lr [3.125e-06]
Epoch[28/50](300/305) || training loss 0.0004391 || training accuracy 100.00% || lr [3.125e-06]

Calculating validation results...
Early Stopping...


In [31]:
class CustomTestDataset(Dataset):
    def __init__(self, df, transform=None):
        super().__init__()
        self.df = df.reset_index()
        self.image_id = self.df['품목일련번호']
        self.transform = transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        image_id = self.image_id[idx]
        image_path = f'/opt/ml/final-project-level3-cv-16/test_data/{image_id}'
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        
        return image

In [32]:
test_batch_size = 1

In [33]:
image_dir = './test_data'
test_list = os.listdir(image_dir)
test_list = sorted(test_list)

In [34]:
test_df = pd.DataFrame(test_list)
test_df.columns = ['품목일련번호']

In [35]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

test_dataset = CustomTestDataset(test_df, transform=transform)

In [36]:
image = next(iter(test_dataset))
image

tensor([[[1.2043, 1.1700, 1.1358,  ..., 0.2967, 0.2282, 0.3138],
         [1.2214, 1.1015, 1.1872,  ..., 0.1768, 0.1597, 0.2453],
         [1.1358, 1.1187, 1.1358,  ..., 0.2282, 0.2796, 0.2796],
         ...,
         [0.6392, 0.6221, 0.5878,  ..., 0.6734, 0.5707, 0.6221],
         [0.4508, 0.4508, 0.5022,  ..., 0.7077, 0.7933, 0.7591],
         [0.5022, 0.5364, 0.5022,  ..., 0.6906, 0.7933, 0.7933]],

        [[1.4307, 1.3782, 1.3431,  ..., 0.3452, 0.2402, 0.3803],
         [1.4482, 1.3256, 1.4132,  ..., 0.2052, 0.1877, 0.2927],
         [1.3431, 1.3081, 1.3431,  ..., 0.2927, 0.3277, 0.3277],
         ...,
         [0.7129, 0.6779, 0.6429,  ..., 0.7654, 0.6429, 0.6954],
         [0.4853, 0.4853, 0.5378,  ..., 0.8179, 0.9230, 0.8880],
         [0.5553, 0.5728, 0.5203,  ..., 0.8004, 0.9230, 0.9230]],

        [[1.5245, 1.5071, 1.4548,  ..., 0.4439, 0.3219, 0.4439],
         [1.5768, 1.4374, 1.5420,  ..., 0.3045, 0.2696, 0.3742],
         [1.4722, 1.4374, 1.4722,  ..., 0.3742, 0.4091, 0.

In [37]:
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=test_batch_size)

In [40]:
model = timm.create_model('resnet18', pretrained=True, num_classes=num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device) 

PATH = f"./results/{name}/best.ckpt"
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

In [41]:
answer= ['nan', 'text', 'mark']

In [49]:
# val loop
with torch.no_grad():
    print("Calculating validation results...")
    model.eval()
    val_loss_items = []
    val_acc_items = []
    for val_batch in test_loader:
        inputs = val_batch
        inputs = inputs.to(device)

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        
        print(answer[int(preds)])     

Calculating validation results...
text
text
nan
nan
mark
nan
nan
nan
nan
mark
mark
mark
nan
nan
nan
mark
nan
text
nan
nan
nan
mark
nan
nan
nan
nan
text
text
text
nan
mark
mark
mark
